In [19]:
import torch

In [20]:
input_size = 4
hidden_size = 4
batch_size = 1

In [21]:
index2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]

one_hot_lookup = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]
print(x_one_hot)

inputs = torch.tensor(x_one_hot).view(-1, batch_size, input_size).float()
print(inputs)

labels = torch.LongTensor(y_data).view(-1, 1)
print(labels)

[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
tensor([[[0., 1., 0., 0.]],

        [[1., 0., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 1.]]])
tensor([[3],
        [1],
        [2],
        [3],
        [2]])


In [22]:
class Model(torch.nn.Module):
  def __init__(self, input_size, hidden_size, batch_size):
    super(Model, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.rnn_cell = torch.nn.RNNCell(input_size=self.input_size, hidden_size=self.hidden_size)
  
  def forward(self, input, hidden):
    hidden = self.rnn_cell(input, hidden)
    return hidden

  def init_hidden(self):
    return torch.zeros(self.batch_size, self.hidden_size)

net = Model(input_size, hidden_size, batch_size)

In [23]:
criterion= torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

In [24]:
for epoch in range(15):
  loss = 0
  optimizer.zero_grad()
  hidden = net.init_hidden()
  print('Predicted string: ', end='')
  for input, label in zip(inputs, labels):
    hidden = net(input, hidden)
    loss += criterion(hidden, label)
    _, idx = hidden.max(dim=1)
    print(index2char[idx.item()], end='')
  
  loss.backward()
  optimizer.step()
  print(', Epoch [%d/15] loss=%.4f' % (epoch + 1, loss.item()))
  

Predicted string: hhhhe, Epoch [1/15] loss=8.3755
Predicted string: hhhhe, Epoch [2/15] loss=6.7752
Predicted string: ohlle, Epoch [3/15] loss=5.5534
Predicted string: ohloo, Epoch [4/15] loss=4.7334
Predicted string: ohloo, Epoch [5/15] loss=4.1789
Predicted string: ohlol, Epoch [6/15] loss=3.7688
Predicted string: ohlol, Epoch [7/15] loss=3.4539
Predicted string: ohlol, Epoch [8/15] loss=3.2071
Predicted string: ohlol, Epoch [9/15] loss=2.9993
Predicted string: ohlol, Epoch [10/15] loss=2.8151
Predicted string: ohlol, Epoch [11/15] loss=2.6564
Predicted string: ohlol, Epoch [12/15] loss=2.5270
Predicted string: ohlol, Epoch [13/15] loss=2.4205
Predicted string: ohlol, Epoch [14/15] loss=2.3240
Predicted string: ohlol, Epoch [15/15] loss=2.2322


In [26]:
class Model(torch.nn.Module):
  def __init__(self, input_size, hidden_size, batch_size, num_layers=1):
    super(Model, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.num_layers = num_layers
    self.rnn = torch.nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers)
  
  def forward(self, inputs):
    hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size)
    outout, _ = self.rnn(inputs, hidden)
    return outout.view(-1, self.hidden_size)


net = Model(input_size, hidden_size, batch_size)

criterion= torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(15):
  optimizer.zero_grad()
  outputs = net(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

  _, idx = outputs.max(dim=1)
  idx = idx.data.numpy()

  print('Predicted: ', ''.join([index2char[x] for x in idx]), end='')
  print(', Epoch [%d/15] loss=%.3f' % (epoch + 1, loss.item()))
  

RuntimeError: 0D or 1D target tensor expected, multi-target not supported